# Exploração dos melhores modelos da Dinâmica Global do Mercado de Diesel


# Setup Inicial

## Bibliotecas Utilizadas

In [1]:
# Manipulação de dados
import numpy as np
import pandas as pd
import json

# Calculo de estatisticas
from scipy import stats

# Manipulação de datas
from time import strftime
from datetime import datetime
from dateutil.relativedelta import relativedelta

# alertas e mensagens de execução
import logging
import warnings


# Prophet para previsão de series de tempo
from prophet import Prophet

# Medir perdormance dos modelos
from sklearn import metrics

#MICE(Multiple Imputation by Chained Equations):
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

## Definições Gerais

In [2]:
# usado para remover as mensagens de output de processamento dos modelos prophet.
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

In [3]:
# definção dos diretórios relativos

# replace with your container name
blob_container_name = 'general' 

# replace with your relative folder path
blob_relative_path_raw = 'raw/mercado_potencial/'
blob_relative_path_enriched = 'enriched/mercado_potencial/'


# replace with your linked service name
linked_service_raw = 'LS_ADLS_RAW_01'
linked_service_enriched = 'LS_ADLS_ENRICHED_01'

ls_raw = mssparkutils.credentials.getPropertiesAll(linked_service_raw)
ls_enriched = mssparkutils.credentials.getPropertiesAll(linked_service_enriched)

converter_dic_raw = json.loads(ls_raw)
converter_dic_enriched = json.loads(ls_enriched)

end_point_raw = (converter_dic_raw['Endpoint'].split("/"))[2]
end_point_enriched = (converter_dic_enriched['Endpoint'].split("/"))[2]


dir_raw = 'abfss://%s@%s/%s' % (blob_container_name, end_point_raw, blob_relative_path_raw)
dir_enriched = 'abfss://%s@%s/%s' % (blob_container_name, end_point_enriched, blob_relative_path_enriched)


In [4]:
# # getting linked services connection string 
# adls_account_name = 'stedlk01dtandev'  
# sas_key = r"9999"

# dir_enriched = 'abfss://general@stedlk02dtandev.dfs.core.windows.net/enriched/mercado_potencial/'
# ls_enriched = 'LS_ADLS_ENRICHED_01'

## Funções Auxiliares

In [5]:
def dividir_train_test(data, num_meses=12):
    '''
    Função que divide os dados a serem usados pelos modelos em treino e teste conforme o
    padrão pd.DataFrame Prothet contendo as colunas ['ds','y']

    Parâmetros:
    ------------
        data: pd.Dataframe
            Dataframe em formato time series do Prothet a ser dividido
        num_meses: int
            Número de meses usados para teste - 12 meses por padrão
    
    Retorno:
    ----------
        train, test: pd.Dataframe
            Dataframes de treino e teste
    '''
    train_data_fim = data['ds'].max() - relativedelta(months=num_meses)
    # divisão dos datasets de Treino: train e Teste:test
    train = data[data['ds'] <= train_data_fim]
    test = data[data['ds'] > train_data_fim]

    return (train, test)

In [6]:
def modeloBase():
    '''
    função modeloBase cria um modelo básico do Prothet

    Parâmetros:
    ------------
        
    Retorno:
    ----------
        model: Prophet model 
            Modelo Prophet a ser ajustado
    '''
    model = Prophet(seasonality_mode='multiplicative',
                    yearly_seasonality=True, 
                    weekly_seasonality=False,
                    changepoint_range=0.8,
                    changepoint_prior_scale=0.05)
    return(model)

In [7]:
def profecias(modelo, dados, treino, teste):
    '''
    função professias ajusta o modelo do Prothet

    Parâmetros:
    ------------
        modelo: modelo Prothet
            Modelo a ser ajustado (.fit)
        dados: pd.DataFrame
            Dados a serem ajustados no formato prophet
        treino:pd.DataFrame
            Dados de treino a serem ajustados no formato prophet
        teste: pd.DataFrame
            Dados de teste a serem ajustados no formato prophet
    Retorno:
        predict: pd.DataFrame
            Dataframe com os valores previstos 
    '''
    modelo.fit(treino)
    future = modelo.make_future_dataframe(periods=len(teste), freq='MS')
    future = pd.merge(future, dados, on='ds', how='inner')
    future = future.fillna(method='ffill')
    
    np.random.seed(1000)

    predict = modelo.predict(future)
    
    return(predict)

In [8]:
def metricas_profecias(y_teste, y_prev, saida='dict', nm_modelo=''):
    '''
    Calcula as seguintes métricas :
        - MSE, MAE, RMSE, MAPE e R²

    Parâmetros:
    ------------
        y_real: float64
            Valores reais de Y do banco de teste 
        y_prev: float64
            Valores previstos de Y, informar pd.DataFrame ou Series contendo ['yhat'] que a função resgata a parte equivalente ao teste
        saida: string
            Formato da saída [print ou dicionario]
        nm_modelo: string
            Nome do modelo

    Retorna:
        metricas: dict ou print
            MSE, MAE, RMSEe MAPE do modelo
    '''
    try:
        if isinstance(y_prev, pd.DataFrame):
            y_prev = y_prev['yhat'][-len(y_teste):]
        elif isinstance(y_prev, pd.Series) and y_prev.name == 'yhat':
            y_prev = y_prev[-len(y_teste):]
    except:
        return (print('Parâmetros fora do padrão!\n'), help(metricas_profecias))

    metricas = {'MSE':metrics.mean_squared_error(y_teste, y_prev),
                'MAE':metrics.mean_absolute_error(y_teste, y_prev),
                'RMSE':np.sqrt(metrics.mean_squared_error(y_teste, y_prev)),
                'MAPE':metrics.mean_absolute_percentage_error(y_teste, y_prev)}
                    
    if saida == 'dict':
        return(metricas)
    elif saida == 'print':
        print('-'*40)
        print('          Métricas de Avaliação')
        print(nm_modelo)
        print('-'*40)
        print(' • MSE : {:.5f}'.format(metricas['MSE']))
        print(' • MAE : {:.5f}'.format(metricas['MAE']))
        print(' • RMSE: {:.5f}'.format(metricas['RMSE']))
        print(' • MAPE: {:.5f}'.format(metricas['MAPE']))
    else:
        return(print('Parâmetros fora do padrão!\n'), help(metricas_profecias))
    

In [9]:
def melhor_metrica (x):
    """Auxilia na viaualização do melhor modelo

        Apresenta o melhor modelo - o de menor erro absoluto percentual (MAPE)

    Parâmetros:
    ----------
        x: pd.DataFrame
            Dataframe com as metricas dos modelos
            
    Retorno:
    ----------
        print do melhor modelo suas métricas e regressores utilizados no modelo
    """
    a = x[x.MAPE == x.MAPE.min()].copy()
    a = a.iloc[0,:]
    print('                   MELHOR MODELO')
    print('Critério: Menor Erro Percentual Absoluto Médio (MAPE)')
    print('-'*60)
    print('Modelo     : '+ a[0])
    print('Regressores: '+', '.join(a[1]))
    print('Metricas:')
    print(a[2:])
    print('-'*60)

In [10]:
def metricas_modelos(dados, treino, teste):
    """Metricas dos modelos 

        Calcula as principais métricas dos modelos de previsão

    Parâmetros:
    ----------
        dados: pd.DataFrame
            Dados a serem ajustados no formato prophet
        treino:pd.DataFrame
            Dados de treino a serem ajustados no formato prophet
        teste: pd.DataFrame
            Dados de teste a serem ajustados no formato prophet
            
    Retorno:
    ----------
        list_result: pd.DataFrame
            Dataframe com as métricas de todos os modelos ajustados
            Métricas retornadas: MSE, MAE, RMSEe MAPE

    """
    dic_result = {'modelo': 0, 'regressores':'' , 'metricas':{'MSE':0,'MAE':0,'RMSE':0,'MAPE':0,'R2':0}}
    list_result = []
    vol = dados[dados.columns[1]]
    dados.drop(dados.columns[1], axis=1, inplace=True)
    regressores = dados.columns.to_list()[2:]       # os 2 primeiros são DataTime e o Y da serie temporal

    for indice, regressor in enumerate(regressores):    
        mod = modeloBase()
        for i in range(indice):                         # passeia por cada lista agregativa
            mod.add_regressor(regressores[i], standardize=False)
        dic_result['modelo'] = 'modelo_'+str(indice)
        dic_result['regressores'] = list(mod.extra_regressors.keys())
        
        forecast = profecias(mod, dados, treino, teste)
        dic_result['metricas'] = metricas_profecias(teste['y'], forecast)
        
        list_result.append(dic_result.copy())
    return(list_result)

In [11]:
def imputar_dados2 (dim):
    '''Imputação de dados

        Imputação de dados faltantes que ainda não foram disponibilizados pelas fontes geradoras.
        A imputação é feita por meio da média móvel (janela=6).

    Parâmetros:
    ----------
        df: pd.DataFrame
            Dados originais       
    Retorno:
    ----------
        df: pd.DataFrame
            Dados com os dados imputados.     
    '''

    lr = LinearRegression()
    imp = IterativeImputer(estimator=lr, missing_values=np.nan, max_iter=10, verbose=2, imputation_order='roman', random_state=0)
    df = dim.copy()
    aux = df.select_dtypes(include=['float64'])
    result = pd.DataFrame(imp.fit_transform(aux), columns=aux.columns)

    for i in df.columns[df.isnull().any(axis=0)]:
        df[i] = np.where(df[i].isnull(), result[i], df[i])

    return(df)

In [12]:
def imputar_dados (df, metodo='mediamovel', janela=6):
    '''Imputação de dados

        Imputação de dados faltantes que ainda não foram disponibilizados pelas fontes geradoras.
        A imputação é feita por meio da média móvel (janela=6).

    Parâmetros:
    ----------
        df: pd.DataFrame
            Dados originais
        método: String
            métodoo de imputação dos dados podendo ser "mediamovel" (default), "media", "ultimo" 
        janela: Int
            Janela de dados utilizados na construção da média móvel. 
            6 meses - default.            
    Retorno:
    ----------
        df: pd.DataFrame
            Dados com os dados imputados.     
    '''

    df = df.drop('ds', axis=1)
    df['temp']=0

    if metodo == 'media':
        df = df.fillna(df.mean())
    elif metodo == 'ultimo':
        df = df.fillna(method='ffill') # Fill ultimo
    elif metodo =='mediamovel':
        for i in df.columns[df.isnull().any(axis=0)]:
            df['temp'] = df[i].rolling(janela ,center=True,min_periods=1).mean()
            df[i] = np.where(df[i].isnull(), df['temp'], df[i])
    else:
        print('metodo não existente!')
        help(imputar_dados)

    #df.reset_index(inplace=True, drop=False)
    df.rename(columns={'yhat':'prev_veget'}, inplace=True)
    df = df.drop('temp', axis=1)
    df.insert(0, 'ds', df.index)
    return(df)

# Ajuste dos Dados

## Resgate dos dados a serem usados nos modelos

In [13]:
ibge_mes   = pd.read_parquet(dir_enriched + 'ibge/IBGE_mes.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})
diesel_mes = pd.read_parquet(dir_enriched + 'anp/anp_diesel_mes.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})
bacen_mes  = pd.read_parquet(dir_enriched + 'bacen/BACEN_mes.parquet',
                                storage_options = {'linked_service' : linked_service_enriched})

## Troca de base do indice de volume de vendas (de jan/2012=100 para jan/2018=100)


In [14]:
CF_newbase = diesel_mes['2018-01-01':'2018-01-01']['Indice_ConsumidorFinal'].values[0]
TRR_newbase = diesel_mes['2018-01-01':'2018-01-01']['Indice_TRR'].values[0]

diesel_mes['Indice_ConsumidorFinal'] = diesel_mes['Indice_ConsumidorFinal']/CF_newbase*100
diesel_mes['Indice_TRR'] = diesel_mes['Indice_TRR']/TRR_newbase*100

## Agregação e ajustes básicos no dados

In [15]:
dinamica = pd.concat([diesel_mes, ibge_mes,bacen_mes],axis=1)

# AJUSTES DO DFs
dinamica = dinamica['2012-06':] # Data inicio em 2012-06 devido a PNAD_Desocupaçao não possuir dados anteriores
dinamica.columns = dinamica.columns.str.replace('consumidor final', 'Volume_ConsumidorFinal')
dinamica.columns = dinamica.columns.str.replace('trr','Volume_TRR')
dinamica.columns = dinamica.columns.str.replace('data', 'ds')

# Linhas não executadas para poder fazer a imputação dos meses rescentes via védia móvel
#dinamica = dinamica[dinamica['Volume_ConsumidorFinal'].notnull()]  # Data fim recorte com base nos dados da ANP
#dinamica = dinamica[dinamica['PIB_Exportacao'].notnull()] # Data fim recorte com base nos dados do IBGE

# definir data final - ultimo mês de dados da ANP - quem define o ultimo mês é a ANP
end = dinamica.loc[pd.isna(dinamica['Volume_ConsumidorFinal']), :].index[0]- relativedelta(months=1)
end = end.strftime('%Y-%m')

dinamica = dinamica[:end]

# primeiro dado nan - mes ainda a ser disponibilizado o pib
try:
    pri = dinamica[dinamica.isnull().T.any()].index[0].strftime('%Y-%m')
except:
    pass
#pri = dinamica.loc[pd.isna(dinamica['PIB_Exportacao']), :].index[0].strftime('%Y-%m') #primeiro nan

In [16]:
dinamica_par = imputar_dados(dinamica, 'mediamovel', 6)             # Dinamica parcial

while len(dinamica_par.columns[dinamica_par.isnull().any(axis=0)])!=0:
    dinamica_par = imputar_dados(dinamica_par, 'mediamovel', 6)     # Dinamica parcial

## Salvando dados da Dinamica Global

In [17]:
dinamica.to_parquet(dir_enriched + 'volumetria/vol04_dinamica_diesel.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

dinamica_par.to_parquet(dir_enriched + 'volumetria/vol04_dinamica_diesel_parcial.parquet', 
                            storage_options = {'linked_service':linked_service_enriched})

##  Separação de Amostras de treino e teste para <br>escolha dos novos modelos a partir de suas métricas

In [18]:
lista_CF  = dinamica.columns[~dinamica_par.columns.str.contains('TRR')]
lista_TRR = dinamica.columns[~dinamica_par.columns.str.contains('ConsumidorFinal')]

df_ConsFinal = dinamica_par[lista_CF]
df_TRR = dinamica_par[lista_TRR]

df_ConsFinal.rename(columns={'Volume_ConsumidorFinal':'y'}, inplace=True)
df_TRR.rename(columns={'Volume_TRR':'y'}, inplace=True)

In [19]:
train_CF, test_CF = dividir_train_test(df_ConsFinal, num_meses=18)
train_TRR, test_TRR = dividir_train_test(df_TRR, num_meses=18)
test_TRR,x = dividir_train_test(test_TRR, num_meses=6) # remvendo os ultimos 6meses do teste para fixar os 12 meses

## Salvando Métricas dos Modelos Ajustados

In [20]:
CF = pd.DataFrame(metricas_modelos(df_ConsFinal, train_CF, test_CF))
TRR = pd.DataFrame(metricas_modelos(df_TRR, train_TRR, test_TRR))

CF  = pd.concat([CF.drop(['metricas'], axis=1), CF['metricas'].apply(pd.Series)], axis=1)
TRR = pd.concat([TRR.drop(['metricas'], axis=1), TRR['metricas'].apply(pd.Series)], axis=1)

In [21]:
CF.to_parquet(dir_enriched + 'volumetria/vol04_metricas_global_ConsFinal.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

TRR.to_parquet(dir_enriched + 'volumetria/vol04_metricas_global_TRR.parquet', 
                      storage_options = {'linked_service':linked_service_enriched})

In [22]:
melhor_metrica(CF)

In [23]:
melhor_metrica(TRR)

# Acesso à Documentação

In [24]:
help(dividir_train_test)

In [25]:
help(modeloBase)

In [26]:
help(profecias)

In [27]:
help(metricas_profecias)

In [28]:
help(metricas_modelos)

In [29]:
help(melhor_metrica)

In [30]:
help(imputar_dados)